In [2]:
import sys
sys.path.append("../decklizer")
from utils import *
from engine import *

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy

from IPython.display import clear_output
import time
import datetime
import re

from scipy.optimize import linprog
from collections import Counter
import itertools

In [2]:
# update front end

# TODO on algorithm
# try a heuristic split method to come up with best 
# 2 layout approach

In [ ]:
data = pd.read_excel('../decklizer/data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                           sheet_name='Schedule')

In [ ]:
df = load_schedule(data,
    customer='P & G',
    technology='SAM',
    color='WHITE',
    cycle='CYCLE 2',                 
)
# df = df.iloc[:10]

In [128]:
B = 4160
w, q, L, n = process_schedule(
    df,
    B=B,
    put_up=17000,
    doffs_in_jumbo=6,
    verbiose=True,
)

The important variables

widths: [154, 158, 170, 208, 218, 234] (mm)
neck in: [4, 4, 5, 7, 7, 7] (mm)
jumbo length (L): 102000 (m)
undeckled jumbos needed (q): [96, 32, 318, 95, 158, 461]


In [17]:
buckets['A']

{'w': [523, 629]}

In [18]:
buckets = dict()
buckets['A'] = dict()
buckets['A']['w'] = [i+j for i, j in zip([205, 195, 220, 160],[3, 2, 3, 1])]
buckets['A']['n'] = [3, 2, 3, 1]
buckets['A']['L'] = 17000
buckets['A']['q'] = [776, 557, 470, 17]
buckets['A']['B'] = 4160

In [3]:
w = [205, 195, 220, 160]
n = [3, 2, 3, 1]
L = 17000
q = [776, 557, 470, 17]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [208, 158, 170, 234]
n = [2, 2, 2, 3]
L = 17000
q = [65, 120, 1001, 100]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [575, 626]
n = [4, 4]
L = 17000
q = [226, 226]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [711, 559, 889, 762, 572]
n = [16, 16, 16, 16, 16]
L = 17000
q = [20, 201, 47, 21, 10]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [208, 234, 170]
n = [2, 3, 2]
L = 17000
q = [805, 437, 226]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [218, 234, 208]
n = [2, 2, 1]
L = 17000
q = [645, 756, 141]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [930, 588, 777, 610]
lm = [8130330, 1152978, 6067685, 495868]
n = [16, 16, 16, 16]
w = [i+j for i,j in zip(w,n)]
B = 3950
L = 34000
q = [math.ceil(i/L) for i in lm]

w = [523, 629]
n = [7, 9]
L = 9900
q = [238, 449]
B = 4144

In [4]:
max_combinations=2
max_patterns=2

layout pattern: {516: 3, 620: 4}
pattern loss: 1.42 %
minimum doffs to fill order: 113
inventory created: {516: 101, 620: 3}

3 possible max 2 combinations
7 unique patterns found
7 possible max 2 patterns

total loss:      3.04 % (1.27e+05 sqm)
total inventory: 0.68 % (2.86e+04 sqm)

inventory created: {516: 2.0, 620: 3.0}

layout summary:

loss: 8.93% 	 22.0 x	 {516: 0, 620: 6}
loss: 1.42% 	 80.0 x	 {516: 3, 620: 4}

total jumbos: 102.0 (4.18e+06 sqm)


In [25]:
combination_params = [2,3]
pattern_params = [2,3,4]
goal = 3
verbiose = True
bucket = buckets['A']
res = []
for max_combinations in combination_params:
    for max_patterns in pattern_params:
        patterns, layout = seed_patterns(bucket['w'], 
                                         bucket['q'], 
                                         bucket['B'], 
                                         bucket['n'], 
                                         max_combinations=max_combinations, 
                                         goal=goal, 
                                         verbiose=verbiose)

        loss, inv, summary = find_optimum(patterns, 
                                          layout, 
                                          bucket['w'], 
                                          bucket['q'], 
                                          bucket['B'], 
                                          bucket['n'], 
                                          bucket['L'],
                                        max_combinations=max_combinations,
                                        max_patterns=max_patterns,
                                        prioritize='time')
        res.append([loss, inv, summary, max_combinations, max_patterns])

layout pattern: {205: 8, 195: 6, 220: 5, 160: 1}
pattern loss: 0.91 %
minimum doffs to fill order: 97
inventory created: {205: 0, 195: 25, 220: 15, 160: 80}

10 possible max 2 combinations
18 unique patterns found
153 possible max 2 patterns

total loss:      0.63 % (4.48e+04 sqm)
total inventory: 8.25 % (5.84e+05 sqm)

inventory created: {205: 3.0, 195: 151.0, 220: 2.0, 160: 24.0}

layout summary:

loss: 1.13% 	 41.0 x	 {205: 19, 195: 0, 220: 0, 160: 1}
loss: 0.29% 	 59.0 x	 {205: 0, 195: 12, 220: 8, 160: 0}

total jumbos: 100.0 (7.07e+06 sqm)
layout pattern: {205: 8, 195: 6, 220: 5, 160: 1}
pattern loss: 0.91 %
minimum doffs to fill order: 97
inventory created: {205: 0, 195: 25, 220: 15, 160: 80}

10 possible max 2 combinations
18 unique patterns found
816 possible max 3 patterns

total loss:      0.46 % (3.02e+04 sqm)
total inventory: 1.64 % (1.08e+05 sqm)

inventory created: {205: 3.0, 195: 14.0, 220: 10.0, 160: 5.0}

layout summary:

loss: 1.42% 	 22.0 x	 {205: 0, 195: 20, 220: 0,

In [27]:
df = pd.DataFrame(res, columns=['loss', 'inventory', 'summary'])

In [33]:
df = df.sort_values('loss').reset_index(drop=True)

In [34]:
df.iloc[0,2]

,loss,jumbos,layout
0,0.552885,20,"{205: 0, 195: 21, 220: 0, 160: 0}"
1,0.144231,4,"{205: 0, 195: 17, 220: 0, 160: 5}"
2,0.0240385,30,"{205: 0, 195: 3, 220: 16, 160: 0}"
3,0,39,"{205: 20, 195: 0, 220: 0, 160: 0}"


In [35]:
res

[[0.6334134615384616,
  {205: 3.0, 195: 151.0, 220: 2.0, 160: 24.0},
         loss jumbos                             layout
  0   1.12981     41  {205: 19, 195: 0, 220: 0, 160: 1}
  1  0.288462     59  {205: 0, 195: 12, 220: 8, 160: 0}],
 [0.459832506203474,
  {205: 3.0, 195: 14.0, 220: 10.0, 160: 5.0},
          loss jumbos                             layout
  0    1.41827     22  {205: 0, 195: 20, 220: 0, 160: 1}
  1   0.264423     41  {205: 19, 195: 1, 220: 0, 160: 0}
  2  0.0240385     30  {205: 0, 195: 3, 220: 16, 160: 0}],
 [0.1328577336641853,
  {205: 4.0, 195: 21.0, 220: 10.0, 160: 3.0},
          loss jumbos                             layout
  0   0.552885     20  {205: 0, 195: 21, 220: 0, 160: 0}
  1   0.144231      4  {205: 0, 195: 17, 220: 0, 160: 5}
  2  0.0240385     30  {205: 0, 195: 3, 220: 16, 160: 0}
  3          0     39  {205: 20, 195: 0, 220: 0, 160: 0}],
 [0.6334134615384616,
  {205: 3.0, 195: 151.0, 220: 2.0, 160: 24.0},
         loss jumbos                    